## Data Visualization with Geopandas and Bokeh

Referencing this tutorial: https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0

In [27]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [4]:
shapefile = '/Users/a666/codeup-data-science/flask-project/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp'

#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['name', 'iso_3166_2', 'geometry']]

#Rename columns.
gdf.columns = ['state', 'state_code', 'geometry']
gdf.head()

,state,state_code,geometry
0,Minnesota,US-MN,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."
1,Montana,US-MT,"POLYGON ((-116.04823 49.00037, -113.05950 49.0..."
2,North Dakota,US-ND,"POLYGON ((-97.22894 49.00089, -97.21414 48.902..."
3,Hawaii,US-HI,"MULTIPOLYGON (((-155.93665 19.05939, -155.9080..."
4,Idaho,US-ID,"POLYGON ((-116.04823 49.00037, -115.96780 47.9..."


Now that I've loaded the geometric data, I'll load the dog- and cat-friendly location lists.

In [18]:
dogs = pd.read_csv('dogs.csv', index_col=0)
cats = pd.read_csv('cats.csv', index_col=0)

In [19]:
dogs

,city,state
0,Tampa,Florida
1,Austin,Texas
2,Las Vegas,Nevada
3,Orlando,Florida
4,Seattle,Washington
5,St. Louis,Missouri
6,Atlanta,Georgia
7,New Orleans,Louisiana
8,Birmingham,Alabama
9,San Diego,California


In [20]:
cats

,city,state
0,Birmingham,Alabama
1,Portland,Oregon
2,Madison,Wisconsin
3,Richmond,Virginia
4,Minneapolis,Minnesota
5,St. Louis,Missouri
6,Tampa,Florida
7,Orlando,Florida
8,Greensboro,North Carolina
9,Denver,Colorado


In [22]:
# Merge dataframes gdf and dogs
merged_dogs = gdf.merge(dogs, left_on = 'state', right_on = 'state')
merged_dogs

,state,state_code,geometry,city
0,Idaho,US-ID,"POLYGON ((-116.04823 49.00037, -115.96780 47.9...",Boise
1,Washington,US-WA,"POLYGON ((-117.03143 48.99931, -117.02665 47.7...",Seattle
2,Arizona,US-AZ,"POLYGON ((-109.04522 36.99991, -109.04367 31.3...",Scottsdale
3,California,US-CA,"POLYGON ((-114.64222 35.05311, -114.62212 34.9...",San Diego
4,Nevada,US-NV,"POLYGON ((-117.02825 42.00002, -114.03422 41.9...",Las Vegas
5,Oregon,US-OR,"POLYGON ((-116.91500 45.99998, -116.67900 45.8...",Portland
6,Missouri,US-MO,"POLYGON ((-89.66292 36.02307, -90.31539 36.023...",St. Louis
7,Oklahoma,US-OK,"POLYGON ((-103.00322 36.99516, -102.04118 36.9...",Oklahoma City
8,Louisiana,US-LA,"POLYGON ((-94.05976 33.01212, -93.09403 33.010...",New Orleans
9,Texas,US-TX,"POLYGON ((-106.50734 31.75429, -106.61953 31.9...",Austin


In [23]:
# Merge dataframes gdf and cats
merged_cats = gdf.merge(cats, left_on = 'state', right_on = 'state')
merged_cats

,state,state_code,geometry,city
0,Minnesota,US-MN,"POLYGON ((-89.95766 47.28691, -90.13175 47.292...",Minneapolis
1,Minnesota,US-MN,"POLYGON ((-89.95766 47.28691, -90.13175 47.292...",St. Paul
2,Washington,US-WA,"POLYGON ((-117.03143 48.99931, -117.02665 47.7...",Seattle
3,California,US-CA,"POLYGON ((-114.64222 35.05311, -114.62212 34.9...",Sacramento
4,Colorado,US-CO,"POLYGON ((-102.05017 40.00081, -102.04012 38.4...",Denver
5,Nevada,US-NV,"POLYGON ((-117.02825 42.00002, -114.03422 41.9...",Reno
6,Oregon,US-OR,"POLYGON ((-116.91500 45.99998, -116.67900 45.8...",Portland
7,Missouri,US-MO,"POLYGON ((-89.66292 36.02307, -90.31539 36.023...",St. Louis
8,Nebraska,US-NE,"POLYGON ((-102.05017 40.00081, -102.05017 40.0...",Omaha
9,Oklahoma,US-OK,"POLYGON ((-103.00322 36.99516, -102.04118 36.9...",Tulsa


Wonderful! All values look correct. Now I need to convert the data to JSON so I can use Bokeh to implement interactivity.

In [25]:
#Read data to json.
merged_dogs = json.loads(merged_dogs.to_json())
merged_cats = json.loads(merged_cats.to_json())
#Convert to String like object.
json_dogs = json.dumps(merged_dogs)
json_cats = json.dumps(merged_cats)

Now I'm ready to render my map. I'll do cats first.

In [ ]:
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_cats)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40)

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(p)